In [1]:
import os
import zipfile
import cjson
import pandas as pd
import xmltodict, json

from xml.dom.minidom import parse

filenames = os.listdir('USA_data')

In [2]:
raw_text = {}

for zipfilename in filenames:
    with zipfile.ZipFile('USA_data/'+zipfilename) as z:
        for filename in z.namelist():
            if not os.path.isdir(filename):
                try:
                    with z.open(filename) as f:
                        raw_text[filename[:-4]] = json.dumps(xmltodict.parse(f))
                except:
                    print filename
                    pass

In [4]:
usa_list = []

for key, val in raw_text.items():
    usa_dict = {}

    value = cjson.decode(val)
    new_val = value['rootTag']['Award']

    usa_dict['id'] = new_val['AwardID']
    usa_dict['title'] = new_val['AwardTitle']
    usa_dict['objective'] = new_val['AbstractNarration']
    usa_dict['state'] = new_val['Institution']['StateCode']
    
    try:
        if type(new_val['ProgramReference']) is dict:
            usa_dict['subjects'] = new_val['ProgramElement']['Text'] + new_val['ProgramReference']['Text']
        elif type(new_val['ProgramReference']) is list:
            text = ''
            for i in new_val['ProgramReference']:
                text = text + ' ' + i['Text']

            usa_dict['subjects'] = text
        else:
            pass
    except:
        pass


    try:
        if type(new_val['FoaInformation']) is dict:
            usa_dict['foa'] = new_val['FoaInformation']['Name']
        elif type(new_val['FoaInformation']) is list:
            text = ''
            for i in new_val['FoaInformation']:
                text = text + ' ' + i['Name']

            usa_dict['foa'] = text
        else:
            pass
    except:
        pass

    usa_list.append(usa_dict)

In [6]:
dfUSA = pd.DataFrame(usa_list)
dfUSA.head(20)

,foa,id,objective,state,subjects,title
0,NaN,9908343,9908343<br/>Ward<br/><br/>This project entitle...,MN,NaN,Directed Templating and Synthesis of Low-Densi...
1,NaN,1231468,Intellectual Merit: There is much to be learne...,NH,NaN,Using Next Generation Sequencing to Quantify t...
2,NaN,1039870,With this award from the Major Research Instru...,PA,MAJOR RESEARCH INSTRUMENTATION CHEMICAL INSTR...,MRI: Acquisition of a New 500 MHz NMR Console ...
3,NaN,1231467,This award focuses on providing support to ena...,MI,NaN,International Medical Geography Symposium; Eas...
4,NaN,1231461,Objective: <br/>The objective of this program ...,PA,Wireless comm & sig processing Computat syste...,Collaborative Research: CCSS: Cyber-Enabled Sm...
5,NaN,1042276,This project will conduct innovative research ...,PA,Cyber-Human Systems (CHS)EAGER,EAGER: Creating a Book Citation Index
6,NaN,1014801,This project synthesizes and advances understa...,DC,NaN,The Impact of Copyright Policy on Innovation i...
7,NaN,1018691,Seamless understanding of the meaning of visua...,NM,NaN,Collaborative Research: RI: Small: A Scalable ...
8,NaN,1046036,This Small Business Innovation Research (SBIR)...,PA,NaN,SBIR Phase I: Artificial Intelligence Software...
9,Other Applications NEC,0520468,Proposal Number: DEB-0520468<br/>Proposal Titl...,PA,NaN,EID: Parasite Induced Susceptibility and Trans...
